# Sequence alignment

Sequence alignment is the process of mapping the elements of one sequence to the corresponding elements of one or more other sequences to compare them. It is a fundamental task for computational biologists, because of its many applications. In this tutorial, you will learn how to align 2 sequences and determine how similar they are, which could be an indication of whether they are evolutionarily closely related.

We'll cover:

- Evolution of DNA sequences
- Aligning sequences
- Finding best alignment

## DNA changes during evolution

There are many similarities between different species in nature. They might be obvious - like having wings, which is a characteristic trait of birds, but also many insects. Or they might not be easily observable - like being able to produce the enzyme DNA polymerase, which is necessary to replicate DNA and therefore all cellular organisms have it. However, not all similarities are due to evolutionary relationship between the species. Sometimes, similar traits develop independently (convergent evolution), because of environmental pressures (e.g. being able to fly provides multiple opportunities for finding food and safe shelter). In this case, the similar traits are called analogous. When different species share traits, because they have a common ancestor, these traits are **homologous**.

<a href="http://group4cladistics.weebly.com/analogous-and-homologous-traits.html"><img src="images/structures.jpeg" alt="Structures - analogous vs homologous" width="700" align="center"></a>

**Sequence homology** is present when sequences from different species:
1. are more similar than would be expected by chance (statistical methods applied)
2. have a common origin

Consider the diagram below. We have a short DNA sequence that is present in 3 different viruses that are evolutionarily related. Virus A is a common ancestor of viruses B and C. We can see that the sequence GGCCTGCCG changed only a little during evolution. In this case, the sequences GGCCTGCAG and GGCCGCCG are homologous.

<img src="images/homology.png" alt="Viral homology" width="400" align="center">

But what exactly changed? If we align each of the sequences from viruses B and C to that of virus A, it will look like this:

<img src="images/align.png" alt="Viral sequences alignment" width="300" align="center">

In the first alignment, we can see 8 matches and 1 mismatch - cytosine (C) in the sequence of virus A was replaced with adenine (A) in virus B. In the second alignment, there are 8 matches and 1 gap - thymine (T) was deleted from virus C.

These differences occur due to different mutations. Below is a summary of mutation types.

<a href="https://www.genome.gov/genetics-glossary/Mutation"><img src="images/mutations.jpg" alt="Mutation types" width="600" align="center"></a>

If you need an introduction about how DNA mutations occur and the different types, check out this <a href="https://youtu.be/I16YlE8qTBU">video</a>. In this tutorial, we'll only be concerned with micro mutations (point mutations) - **insertions**, **deletions** and **substitutions** of individual nucleotides. 
                                                    
We can find out how similar two sequences are by aligning them and accounting for possible mutations. Their similarity could be an indication of homology, suggesting relatedness between the organisms of origin. Statistical analysis is applied to determine the likelihood of relatedness.

First, let's learn how to align sequences.


## Representation of sequence alignments

Aligned DNA sequences are writen, so that nucleotides that are being compared are placed under each other. When the bases at a particular position in the alignment are the same they are a **match**. When they are different, which could be a result of substitution, that's a **mismatch**. **Gaps** occur due to deletions or insertions (indels) and are indicated with a dash "-". 

<a href="https://towardsdatascience.com/pairwise-sequence-alignment-using-biopython-d1a9d0ba861f"><img src="images/alignment-events.png" alt="Sequence alignment events" width="400" align="center"></a>

## Types of sequence alignment

There are 2 major types of sequence alignment - **pairwise sequence alignment** (PSA) and **multiple sequence alignment** (MSA). Pairwise sequence alignment can be global or local. **Global** is when sequences are fully compared from end to end. **Local** is when only some parts of the sequences are compared. Multiple sequence alignment is when three or more sequences are compared at once, generating complex outputs (you will learn more about MSA in the next project).

<a href="https://www.researchgate.net/figure/Global-alignment-vs-Local-alignment_fig1_322704711"><img src="images/alignment.png" alt="Sequence alignment" width="500" align="center"></a>

## Possible alignments

If two sequences are the same length and you don't consider the posibility for insertions and deletions, alignment is straightforward. However, in any other case, there are multiple possibilities. How would you align the sequences *AATCT* and *ATGT*?
<img src="images/example-alignments.png" alt="Example alignments" width="500" align="center">
These are just a few options. Many alignments are possible, but a lot of them would be unlikely from biological perspective. So what alignments are we interested in? Let's examine how we can compare sequences in order to decide how to align them.

## Difference between sequences

Different alignments suggest a different number of mutations. If two sequences are close to being identical, because of homology, then we would expect that a small number of mutations have turned one of the sequences into the other. 
How can you calculate how many mutations are necessary to turn one sequence into another?

If you consider only substitutions, then you are interested in the <a href="https://en.wikipedia.org/wiki/Hamming_distance">Hamming distance</a> of the sequences - the number of mismatches. Write a script that calculates the Hamming distance of 2 sequences with equal length. Example:

**Input**: <br>
ATGTCA <br>
AGGTGA <br>
**Output**: 2

In [ ]:
seq1 = input()
seq2 = input()

def hamming_distance(s1,s2):
    D = 0
#   Your code
#   Hint: You could use the zip function to group corresponding characters from the 2 strings.
    return D

print(hamming_distance(seq1,seq2))

If you consider both substitutions and indels, then you'll want to know the **edit distance** (<a href="https://en.wikipedia.org/wiki/Levenshtein_distance">Levenshtain distance</a>) of the sequence - the minimum number of operations required to turn one sequnce into the other. The edit distance is not as trivial to compute as is the Hamming distance. Why? 

In the section about possible alignments, you probably noticed that there are multiple possible positions for indels in both strings. Theoretically, you could have infinite number of indels starting from any given position of either sequence. However, generating a lot of these alignments will be redundant when trying to find edit distance. For example, notice that the edit distance couldn't be bigger than the number of bases, because in the worst case scenario you would just substitute each base. In fact, it cannot be bigger than Hamming distance. So there is no point in introducing more indels than the number of mismatches. Even considering this, the number of possible alignments rises very quickly with the increase in sequence length. Implementing naive or brute force approach in this case is definitely not optimal. So what could you do? 

### Dynamic programming approach

According to Bellman's principle of optimality, optimal solutions consist of optimal partial solutions. This means
that you can build the solution of your problem from the solutions of several smaller problems. 

Imagine that you want to find the length of the shortest path from Sofia to Sunny Beach. Let's say that you know the distances between cities connected by major roads. There are 2 major routes from Sofia to the Black Sea - through Varna and Burgas.

<img src="images/dynamic.png" alt="Dynamic problem example" width="500" align="center">

The minimal distance from Sofia to Sunny Beach is the minimum of the following 2 values - the road distance from Sofia to Varna (on Hemus Highway) plus the minimal distance from Varna to Sunny Beach and the road distance from Sofia to Burgas (on Trakia Highway) plus the minimal distance from Burgas to Sunny Beach. 

<code>min_dist(Sofia, Sunny Beach) = min (road_dist(Sofia, Varna) + min_dist(Varna, Sunny Beach), road_dist(Sofia, Burgas) + min_dist(Burgas, Sunny Beach))</code>

In order to solve the problem for Sofia, you need to solve the same problem for Varna and Burgas first. Now, it becomes particularly easy, because there's a direct link between Varna and Sunny Beach (on road E87) and between Burgas and Sunny Beach (on road E87), so you just take these distances. 

<code>min_dist(Varna, Sunny Beach) = road_dist(Varna, Sunny Beach)</code><br>
<code>min_dist(Burgas, Sunny Beach) = road_dist(Burgas, Sunny Beach)</code>

Work your way backwards from the destination and at each step calculate the minimum distance for each city in the current layer of cities. Save the current values in a list. You end up with the value for the full route.

<img src="images/tables.png" alt="Dynamic problem example tables" width="500" align="center">

Obviously, this example is oversimplified. But the same approach can be applied for much bigger graphs (maps) with many more edges (roads) and nodes (cities), where it is not so trivial to find the answer. If this is the first time you hear about graphs or you need to refresh your knowledge, I recommend this <a href="https://youtu.be/LFKZLXVO-Dg">introduction to graph theory</a>. Then, watch this <a href="https://youtu.be/pVfj6mxhdMw">explanation of Dijkstra's algorithm</a> for shortest path problem to understand better what the method is all about and how it works. Also, check out this <a href="https://youtu.be/aPQY__2H3tE@">dynamic programing</a> video to see some more interesting problems and tips about how to approach them.

### Calculating edit distance

Can we apply dynamic programming to find edit distance of 2 sequences? Let's say we want to find the edit distance of the sequences *ATG* and *TGT*. Thinking backwards, is there any form of a single step that can be applied to  partial solutions to get the final answer? Assume that we already know the edit distance of alignments of subsequences (without the last nucleotide).

- ATG|TG - we can get from any alignment of these sequences to an alignment of ATG|TGT by simply inserting a T at the end of the second sequence =>

<code> edit_distance(ATG,TGT) <= edit_distance(ATG,TG) + 1 </code>

- AT|TGT - we can get from any alignment of these sequences to an alignment of ATG|TGT by simply inserting a G at the end of the first sequence =>
    
<code> edit_distance(ATG,TGT) <= edit_distance(ATG,TG) + 1 </code>
    
- AT|TG - if we add a nucleotide at the end of each sequence of any alignment of these sequences, we get an alignment of the new sequences that implies either the exact same number of operations to transform one sequence into the other (if the new nucleotides are the same) or the same number of mutations plus one (substitution, if the nucleotides are different) =>
    
<code> edit_distance(ATG,TGT) <= edit_distance(AT,TG) + is_not_equal(G, T) </code>

Think carefully about the statements above. Try to visualize what they mean. Drawing could help. 
    
We could use a table like the one below, in which each cell corresponds to a combination of some substrings of the two sequences to represent alignments of subsequences. The value of the cell is the minimum edit distance for the corresponding subsequences. We could immediately initialize the values that we know and then calculate the rest. Try to figure out the rules for filling the table.

<img src="images/edtable.png" alt="Edit distance table" width="500" align="center">

Watch the following videos about the <a href="https://youtu.be/8Q2IEIY2pDU">edit distance problem</a> and the <a href="https://youtu.be/0KzWq118UNI">dynamic solution</a> and then write your own implementation.

In [ ]:
import numpy as np

seq1 = input()
seq2 = input()

def delta (s1, s2, i, j):
#    ...

def edit_distance(s1,s2):
    m = len(s1) + 1
    n = len(s2) + 1
    edmat = np.zeros((m,n))
#   Your code ...
#   print(edmat)
    return edmat[m-1][n-1]

print(edit_distance(seq1,seq2))

### Finding the alignment - backtrace

We calculated the edit distance of sequences, but how do we know which of all possible alignments of the sequences has the minimum edit distance? For this we need to trace back our way through the edit distance matrix. We start at the lower right corner and we move in the direction of the minimum edit distance that we derived the current value from. Based on the direction that we are moving in, we can determine the type of mutation that occured at the given position. For the ATG|TGT the backtrace will look like this:

<img src="images/backtrace.png" alt="Backtrace" width="600" align="center">

Write a function <code>backtrace()</code> that recursively traces back from the scores in the matrix and returns an optimal score alignment of the sequences (the sequences with gap symbols at the relevant positions). For example:

**Parameters**: <br>
seq1 = "TGT" <br>
seq2 = "ATG" <br>
edmat = [[0,1,2,3],[1,1,1,2],[2,2,2,1],[3,3,2,2]] <br>
**Returned value**: <br>
['ATG-','-TGT']

The solution might not be unique. There could be multiple alignments with the same optimal score. For the purposes of this exercise, just output one of those optimal alignments. If this is the first time you hear about recursion, watch this <a href="https://youtu.be/rf60MejMz3E">brief explanation</a>.

In [ ]:
seq1 = "TGT"
seq2 = "ATG"
edmat = [[0,1,2,3],[1,1,1,2],[2,2,2,1],[3,3,2,2]] 

def backtrace(i,j,al_seq1,al_seq2):
#    return backtrace(i,j-1,al_seq1,al_seq2)
#    return backtrace(i-1,j,al_seq1,al_seq2)
#    return backtrace(i-1,j-1,al_seq1,al_seq2)
#    return [al_seq1[::-1],al_seq2[::-1]]
    
print(backtrace(len(seq1),len(seq2),"",""))

## Scoring alignments

Great, so we are able to determine the number of mutations that separates two sequences. This information could be indicative of the likelihood that one of the sequences originated from the other, but is not sufficient to determine homology with high accuracy and certainty. Why? 

Because in nature, certain events are more likely than others. For example, indels are usually less likely than substitutions. Furthermore, deletions tend to occur more often than insertions and certain types of substitutions are more likely than others (e.g. C could be substituted more for T than for A). These likelihoods depend on the organism, nucleotide location and many other factors. Therefore, to make a more biologically relevant alignment, we need to give different weight to different types of mutations. These weights are determined based on probabilities that are derived experimentally and are problem specific. <a href="https://en.wikipedia.org/wiki/Substitution_matrix">Substitution matrix</a> and other information can be used to generate a scoring scheme. For example, we could say that substituting C for A will cost 1, any other substitution will cost 2, insertions will cost 3 and deletions - 4. In many cases opening a gap (due to deletion) somewhere in the sequence costs more than extending one. This is called <a href="https://en.wikipedia.org/wiki/Gap_penalty#Types"> affine gap penalty</a>. For example, if the sequence *ATG* has 2 gaps introduced like *A-T-G*, the overall cost is higher, than if they are placed like *A--TG*, because empirical evidence shows that having one big deletion is more likely than two smaller ones in the same sequence.

The two major components of scoring an alignment are:
- **Identity score** - sum of matches and mismatches (multiplied by negative coefficient = cost)
- **Gap penalty** - function of gaps (representing indels)

The best (most likely) alignment of any two sequences is considered to be the one with maximum score, according to the scoring rules.

<a href="https://www.ncbi.nlm.nih.gov/books/NBK62051/"><img src="images/blast-scoring.jpg" alt="Scoring alignments" width="500" align="center"></a>

Check out slides 13-15 from <a href="https://www.ncbi.nlm.nih.gov/CBBresearch/Przytycka/download/lectures/PCB_Lect02_Pairwise_allign.pdf">this presentation</a> to see examples of how alignments could be scored.

## Algorithms for pairwise sequence alignment


### Sequence alignment - steps

1. Input sequences
2. Define scoring scheme for possible alignments, considering:
    * Matches and mismatches (costs for substitutions)
    * Gaps (effects of insertions and deletions)
3. Build score matrix
4. Find highest scoring alignment by backtracing

Here's an <a href="https://youtu.be/-bjSPP2v6_Q"> overview</a> of algorithms for global and local alignment.

### Global alignment (Needleman-Wunsch algorithm)

You have already done end-to-end comparision of sequences. The only difference is that now you have to apply a scoring scheme to make it more biologically relevant. Finding the optimal sequence with different costs for different operations is essentially just a modification of the solution for the edit distance problem that you have implemented. Here are videos of the <a href="https://youtu.be/of3B02hZGS0">full Needleman-Wunsch algorithm</a> and <a href="https://youtu.be/BGV-hUoHF9k">an example implementation</a>.

Write your own implementation of this algorithm. You can use your edit distance code and modify it to include penalties, based on a penalty matrix (make up one). Save your implementation in a file. If you want to submit this file for feedback, include your initials in the filename.

### Local alignment (Smith-Waterman algorithm)

The goal of local alignment is to find parts of the sequences that match. You are interested in subsequences that are identical or close to being identical. Therefore, you have to find the best alignment within the sequences. In this case, it would be helpful to not only penalize (-) mismatches and gaps, but also to reward (+) matches, so you can easily find regions within the score matrix with most matches. This allows you to calculate **similarity score**. The best alignment will end with the highest score, so you can start your backtracing from there and finish when you reach a score of 0. This is the principle of <a href="https://en.wikipedia.org/wiki/Smith%E2%80%93Waterman_algorithm">Smith-Waterman algorithm</a>.

<a href="https://commons.wikimedia.org/wiki/File:Smith-Waterman-Algorithm-Example-Step3.png"><img src="images/sw-bt.png" alt="Smith-Waterman backtracing" width="300" align="center"></a>

### Aligning sequences with Biopython

Versions of these algorithms are implemented in the <code>Align</code> module of Biopython. Check out the <a href="https://biopython.org/docs/1.76/api/Bio.Align.html#submodules">documentation</a> and write a script that outputs all optimal local alignments of 2 sequences (full sequences printed) and their score, based on the following scoring scheme:

- matches -> +1
- mismatches -> -1
- opening a gap -> -2
- extending a gap -> -1

In [ ]:
from Bio import Align

# Define input sequences. Expected score output with these sequences: 3
seq1 = "ATGTGTAT"
seq2 = "TGTT"

# Initialize aligner
aligner = Align.PairwiseAligner()

# Set aligner mode

# Set scoring scheme parameters

# Align sequences

# Print first best alignment and its score. 
# Hint: You should have a list of only the best alignments with equal scores.

Now, it's time to align your viral sequences! 

Use SeqIO to get input from your file and perform global alignment of the reference genome and each one of the other viral sequences. Use the same scoring scheme as above. For each alignment, record the id of the sequence that you are comparing to the reference genome and the alignment score. Save the results in a csv file called *alignments_scores.csv*. 

It may take 2-5 minutes for the program to finish. If you have problems with computing full values, because it takes too much time or the server crashes, try comparing substrings of the sequences (e.g. first 10000 nucleotides or less).

In [ ]:
from Bio import SeqIO
import pandas as pd

# Initialize lists
id_seq = []
score = []

# Parse the data
ref = SeqIO.read("<filename>", "fasta") # only one record, so it's ok to just read the file
seqs = SeqIO.parse("<filename>", "fasta") # multiple records, so it's useful to parse the file

# Initialize aligner

# Set aligner mode

# Set scoring scheme parameters

# Iterate through records
    # Append ID to list
    # Align sequences
    # Append score to list

# Save to csv
data = {'ID':id_seq,'Score':score}
df = pd.DataFrame(data=data)
df.to_csv('alignments_scores.csv')

How similar do these viruses seem to be to SARS-CoV-2? You should be able to see now that the alignment with genomic sequence with ref. ID *MN996532.1* has the highest similarity score compared to alignments with other viral sequences on the list.

Of course, the results that we are getting are dependent on our scoring scheme. Using reference information, such as substitution matrices, we might be able to make more accurate comparisons. But how certain are we that the scores that we are getting are a result of homology and not just random chance? Explore this question in the next tutorial - T4 - Similarity statistics.